In [19]:
import os
import time
import sys
import getpass
from drugrelink import pairs
import pandas as pd

In [20]:
print(sys.version)

3.7.1 (default, Dec 14 2018, 13:28:58) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


In [21]:
print(time.asctime())

Sat Sep 28 17:59:47 2019


In [22]:
print(getpass.getuser())

work


In [23]:
from drugrelink import pairs
df = pairs.data_non_overlap(validation_path='/home/lxu/drugrelink/data/validation-statuses.tsv',train_path='/home/lxu/drugrelink/data/transformed-features.tsv.bz2',symptomatic_path='/home/lxu/drugrelink/data/probabilities.tsv')

In [24]:
print(len(df.loc[df['label']==0]),len(df.loc[df['label']==1]))
print(df.head())
print(df['compound'][1])

202313 6855
   Unnamed: 0           compound              disease  label
0           0  Compound::DB01048  Disease::DOID:10652      0
1           1  Compound::DB05812  Disease::DOID:10652      0
2           2  Compound::DB00659  Disease::DOID:10652      0
3           3  Compound::DB00284  Disease::DOID:10652      0
4           4  Compound::DB01193  Disease::DOID:10652      0
Compound::DB05812


In [25]:
nodes_df = pd.read_csv('/Users/work/Documents/GitHub/drugrelink/data/nodes.tsv',sep='\t')
nodes_list = nodes_df['id'].tolist()
names_list = nodes_df['name'].tolist()
nodes_dict = dict(zip(nodes_list, names_list))

In [26]:
import pickle
from node2vec.edges import HadamardEmbedder
with open('/Users/work/Documents/GitHub/drugrelink/resources/predictive_model/edge2vec/0/word2vec_model.pickle', 'rb') as file:
    word2vec = pickle.load(file)
def vectors(df):
    vectors=[]
    for _,row in df[['compound','disease']].iterrows():
        c=row['compound']
        
        d=row['disease']
        
   
        edges_embs = HadamardEmbedder(keyed_vectors=word2vec.wv)
        vector = edges_embs[(c,d)]
        vectors.append(vector)
    return vectors

In [27]:
positive = df.loc[df['label']==1]
print(len(positive))
negative = df.loc[df['label'] == 0]
print(len(negative))
new_negative = negative.drop(df.loc[(df['compound']=='Compound::DB02546') & (df['label']== 0)].index)
train_negative = negative.sample(n=len(positive))
hdac6_df= df.loc[(df['compound']=='Compound::DB02546') & (df['label']== 0)]
alzheimer_df = df.loc[(df['disease']=='Disease::DOID:10652') & (df['label']==0)]

6855
202313


In [13]:
from sklearn.linear_model import LogisticRegressionCV
import numpy as np

def retrain(X,y):
    model = LogisticRegressionCV().fit(X,y)
    return model


# Prediction for HDAC6

In [37]:
X = vectors(positive) + vectors(train_negative)
y= positive['label'].tolist() + train_negative['label'].tolist()
model = retrain(X,y)
predict_x = vectors(hdac6_df)

/Users/work/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2052: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [45]:
import pandas as pd
results = pd.DataFrame({
                      
    'compound_id': hdac6_df['compound'].tolist(),
    'disease_id': hdac6_df['disease'].tolist(),
    'previous_class': hdac6_df['label'].tolist(),
    'predict_class': model.predict(predict_x),
    'probability' : model.predict_proba(predict_x)[:,1]   
}, columns = [ 'compound_id','disease_id','previous_class','predict_class','probability'])

In [46]:
cname= []
dname =[]
for _,row in results[['compound_id','disease_id']].iterrows():
    c = nodes_dict[row['compound_id']]
    d = nodes_dict[row['disease_id']]
    cname.append(c)
    dname.append(d)
results.insert(loc=1,column='compound_name',value=cname)
results.insert(loc=3,column='disease_name',value=dname)

In [47]:
results

,compound_id,compound_name,disease_id,disease_name,previous_class,predict_class,probability
0,Compound::DB02546,Vorinostat,Disease::DOID:10652,Alzheimer's disease,0,1,0.566710
1,Compound::DB02546,Vorinostat,Disease::DOID:9206,Barrett's esophagus,0,1,0.741109
2,Compound::DB02546,Vorinostat,Disease::DOID:13241,Behcet's disease,0,0,0.230724
3,Compound::DB02546,Vorinostat,Disease::DOID:11949,Creutzfeldt-Jakob disease,0,0,0.394443
4,Compound::DB02546,Vorinostat,Disease::DOID:8778,Crohn's disease,0,1,0.601205
5,Compound::DB02546,Vorinostat,Disease::DOID:11555,Fuchs' endothelial dystrophy,0,0,0.262356
6,Compound::DB02546,Vorinostat,Disease::DOID:11119,Gilles de la Tourette syndrome,0,0,0.137307
7,Compound::DB02546,Vorinostat,Disease::DOID:12361,Graves' disease,0,0,0.243305
8,Compound::DB02546,Vorinostat,Disease::DOID:2986,IgA glomerulonephritis,0,1,0.798797
9,Compound::DB02546,Vorinostat,Disease::DOID:13378,Kawasaki disease,0,0,0.337389


In [50]:
rr = results.loc[(results['previous_class']==0) & (results['predict_class']==1)].sort_values(by='probability',ascending = False)
rr

,compound_id,compound_name,disease_id,disease_name,previous_class,predict_class,probability
34,Compound::DB02546,Vorinostat,Disease::DOID:219,colon cancer,0,1,0.888278
57,Compound::DB02546,Vorinostat,Disease::DOID:12365,malaria,0,1,0.815486
8,Compound::DB02546,Vorinostat,Disease::DOID:2986,IgA glomerulonephritis,0,1,0.798797
1,Compound::DB02546,Vorinostat,Disease::DOID:9206,Barrett's esophagus,0,1,0.741109
12,Compound::DB02546,Vorinostat,Disease::DOID:7693,abdominal aortic aneurysm,0,1,0.711235
64,Compound::DB02546,Vorinostat,Disease::DOID:4045,muscle cancer,0,1,0.695527
90,Compound::DB02546,Vorinostat,Disease::DOID:418,systemic scleroderma,0,1,0.688700
89,Compound::DB02546,Vorinostat,Disease::DOID:9074,systemic lupus erythematosus,0,1,0.684227
96,Compound::DB02546,Vorinostat,Disease::DOID:9352,type 2 diabetes mellitus,0,1,0.668103
45,Compound::DB02546,Vorinostat,Disease::DOID:3121,gallbladder cancer,0,1,0.667087


# Prediction for Alzheimers's disease

In [51]:
new_negative = negative.drop(df.loc[(df['disease']=='Disease::DOID:10652') & (df['label']==0)].index)
train_negative = negative.sample(n=len(positive))
#hdac6_df= df.loc[(df['compound']=='Compound::DB02546') & (df['label']== 0)]
alzheimer_df = df.loc[(df['disease']=='Disease::DOID:10652') & (df['label']==0)]

In [52]:
X = vectors(positive) + vectors(train_negative)
y= positive['label'].tolist() + train_negative['label'].tolist()
model = retrain(X,y)
predict_x = vectors(alzheimer_df)

/Users/work/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2052: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [53]:
import pandas as pd
results = pd.DataFrame({
                      
    'compound': alzheimer_df['compound'].tolist(),
    'disease': alzheimer_df['disease'].tolist(),
    'label': alzheimer_df['label'].tolist(),
    'predict_label': model.predict(predict_x),
    'output_std' : model.predict_proba(predict_x)[:,1]   
}, columns = [ 'compound','disease','label','predict_label','output_std'])

In [54]:
cname= []
dname =[]
for _,row in results[['compound','disease']].iterrows():
    c = nodes_dict[row['compound']]
    d = nodes_dict[row['disease']]
    cname.append(c)
    dname.append(d)
results.insert(loc=1,column='compound_name',value=cname)
results.insert(loc=3,column='disease_name',value=dname)

In [55]:
rr = results.loc[(results['label']==0) & (results['predict_label']==1)].sort_values(by='output_std',ascending = False)

In [58]:
rr.to_csv('alzheimer prediction.csv')
#Biperen -> Alzheimer, Imiquimod unknown Alzheimer, 